# 6. FUNCTIONS

[List of q built-in functions](http://code.kx.com/q4m3/A_Built-in_Functions/)

## 6.1. Function specification

- Because a q function can __modify global variables__, q is not a pure functional language.

- Function definition notation: {[param1, param2, ...]expr1;expr2;...}
- if only up to 3 parameters is used x, y, z, you do not have to declare them
- function name is optional: lambda or anonymous function
    - definition {x*x}[3] -> returns 9
- no return statement: the function returns the last expression evaluated
    - finish the function definition with a semi-colon to return nil (::)
    - ; is an opererator that evaluates its left operand and discards its value then evaluates its right operand and returns its value
    - : (return) in a function body terminates the execution of the function early and returns the expression to its right
- a function is a first class value
- a function can be assigned to a variable
- expressions in a function are evaluated from left to right
- valence (rank): number of parameters in a function
    - maximum valence permitted: 8 parameters
        - to exceed this: encapsulate multiple parameters in a list or dictionary
    - niladic function: function with no parameters

In [ ]:
too_many_params:{[q;w;e;r;t;z;u;o;l] (q;w;e;r;t;z;u;o;l)}

___Keep it short___
- Q functions should be compact and modular:
    - each function should perform a well-defined unit of work.
- Due to the power of q operators and built-in functions, many functions are one line.
- When a function exceeds 20 statements, you should look to refactor it.

___q is NOT ambivalent___ depending on their valence:
- meaning
    - q operators and built-in functions are __not overloaded on valence__, meaning that the same function does not have different behavior in a monadic and dyadic form. HOWEVER:

___q IS ambivalent___ in terms of several other aspectcts:
- meaining:
    -  q operators and built-in functions __are overloaded on__:
        - type,
        - sign,
        - rank of arguments,
        - etc.
        - sometimes multiply so.

### 6.1.3. Function application (function call)

- function application is strict: expressions in arguments are evaluated first, substitution happens only afterwards

- method 1:
    - func[arg1;arg2;...;arg8]
- method 2: with juxtaposition
    - func arg1 arg2 ...
- method 3: call function by symbolic name:
    - `func arg1 ...

### 6.1.4. Functions with no return value

- notation: void_function:{expr;} - ends with a semi-colon
- returns the nil item: ::
- purpose of these functions:
    - for only side effects: sending an asynchronous message; updating a global variable

### 6.1.7. Implicit parameters

- __x, y, z__ are the 3 positional parameters defined implicitly inside every function, you can use them without declaring them as parameters

### 6.1.8 Anonymous functions and Lambda expressions

- All q functions are anonymous functions (or lambda expression or lambdas) by defalut
    - until assigned to a variable
- Use cases:
    - in-line macro????
    - __dynamic dispatch__: functions are placed in a collection (e.g., list, dictionary, table) and then selected dynamically (i.e., at runtime) rather than by an intrinsic name

In [ ]:
powers:({1}; {x}; {x*x}; {x*x*x})
powers[3;2]

### 6.1.9. The identity function ::

- an identity function returns its input as its output
- notation: ::[1;32;3] -> returns 1 32 3

### 6.1.10 Functions are data

- data categories in q:
    - atoms
    - lists
    - dictionaries
    - tables
    - __FUNCTIONS__
        - they can be passed around as any other data categories, e.g. as an item in a list or dictionary
        - functions can be arguments of another (higher-order) function or
        - return values

In [ ]:
apply:{x y}
sq:{x*x}
apply[sq; 5]

## 6.2. Call-by-name

__q functions are call by value__
- arguments are reduced to values and copies are made

In [ ]:
a:5
func1:{x-1}
a
func1 a
a

__except when the variable size is too large__
- names of global variables are passed instead of their values
- q kernel handles arguments automatically and decides to call them as values or names
    - built-in functions using call-by-name of global variables:
        - __get__
        - __set__
- The result of any built-in call-by-name function application is the __symbol__ that was passed as input.
    - This enables call-by-name functions to be chained – i.e., composed.

In [ ]:
gv:323
get `gv / the symbolic name should be used as parameter
`gv2 set 101 / a symbolic name should be used as the name of the variable
gv2

## 6.3. Local and global variables

- __Local variable__: assigned with : within a function
- 24 local variables are allowed within a function
- A local variable exists only for the duration of an application. There is no notion of a static local variable that maintains its value across applications.
- A local variable is not visible outside its immediate scope of definition.
- A local variable cannot be used as an argument to a q function that uses call-by-name.
- q does not have lexical scoping: A local variable is not visible within the body of a local function defined in the same scope
    - use projection instead:

In [ ]:
f:{[p1] a:42; helper:{[a; p2] a*p2}[a;]; helper p1}
f 2

In [ ]:
f:{[p1] a:42; helper:{[a;p2] a*p2}[a;p1]}
f 2

__Global variable__
- every variable assigned outside of any function definition is a global variable
- global variables are visible inside any function body
- 32 is the maximum number of global variables that can be referenced inside a function
    - to circumvent this, use lists or dictionaries (dictionaries provide pass by name)
- you can modify or declare a global variable from inside a function
    - `a set 42
    - you can use ::, but it only works for declaring a global variable. if there already exists a global variable, a local variable is created. avoid this usage as it might be confusing if we accidentally choose a name that is already declared.

## 6.4. Projection

- __Projection__: specifying only some of the parameters in function application, which results a function with the remaining parameters

### 6.4.1. Function projection

- projecting a function onto the remaining arguments means: when a portion of arguments in a function is fixed and the remaining ones vary, you can do that to specify only the fixed arguments and leave the specification of the varying ones for later. (like the .bind() method in js)
- notationally, projection is a partial function application, in which some arguments are supplied (the fixed one) and others (the varying ones) are omitted
- the result is a partial function application
- you can assign the partial application of a function to a variable and then use it as a monadic function in case of the example
- projection is related to _currying_
    - Both provide realizations of the mathematical observation that __a function of two parameters is equivalent to a function of the first parameter that returns a function of the second parameter__.

In [ ]:
add:{x+y}
add[42;] / partial application
add[42;][3] / projected application
pa:add[42;]
pa[3]

- projecting out successive parameters:

In [ ]:
add3:{x+y+z}
add3[2;;][3;][4] / is the same as:
add3[2][3][4] / do not use this notation as it obscures original intent of the code
9

!!! __important__
- The function body in a projection is captured at the time it is first encountered by the interpreter.
- Thus if the underlying function is subsequently changed, __the projection is not__.

### 6.4.2. Operator projection

- a dyadic q operator can be projected by fixing one of its operand
    - you can do the in an infix and a prefix form
        - infix form using parenthesis (only the left one can be fixed due to ambiguity): (7*)[6]
        - prefix form using square brackets (either one of the operands can be fixed): *[7;][6] or *[;6][7]

In [ ]:
7*6
(7*)[6] / or (7*) 6
mult:(7*)
mult 6

In [ ]:
*[7;6] / using the multiplication operator as a function
*[7;][6] / multiplication operator projected onto its second operator by fixing the first one
*[;6][7] / multiplication operator projected onto its first operator by fixing the second one

### 6.4.3. Multiple projections

- mutliple projections can be applied to functions having more than two parameters
- these functions can be projected in multiple ways
    - in one step: provide all arguments and omit only one, and project the function to its omitted parameter
    - in multiple steps: provide the arguments to be fixed one by one in separate steps until you reach the that you want your function to be projected onto

In [ ]:
{x+y+z}[1;;3]
{x+y+z}[1;;3] 2 / project in one step

In [ ]:
 / project in two steps
{x+y+z}[1;;] / first step
{x+y+z}[1;;][;3] / second step
{x+y+z}[1;;][;3] 2 / call the function with the varying parameter

## 6.5. Everything is a map - the relationship between q data structures and functions

### 6.5.1. Similarity of notation

- a list is a map: defined by positional retreival via item indexing. In a list of non-negative integer numbers, i.e. the indices are the keys and the items are the values
- a dictionary is a map defined by a key-value lookup
- a function is a map defined by a sequence of expressions representing an algorithm for computing an output value from the input:
    - in a function the output values are mapped to their input values (arguments are the key, the return value is the value)

### 6.5.2 List of Indices, Keys and Arguments

- that is why you can use a list of indices to get several items in a list or
- a list of keys to get a list of values from a dictionary or
- a list of arguments to get a list of return values from a function
- examples:

In [ ]:
L:10 20 30 40 50
L[2 4]
d:`a`b`c!10 20 30
d[`a`c]
f:{x*x}
f[2 5]

- parameter lists are examples of compositions of maps:

In [ ]:
/
  I     L
--------------
0 |-> 2 |-> 4
1 |-> 5 |-> 25

  ks     d
---------------
0 |-> `a |-> 10
1 |-> `c |-> 30

  I     f
--------------
0 |-> 2 |-> 4
1 |-> 5 |-> 25
\

### 6.5.2. Indexing at depth

- the relationship between indexing at depth and multivalent function evaluation
- a nested list:
    - notationally, is a lists of lists: nested_list[0][4][2]
    - functionally, can be viewd as a multivariate positional retreival: nested_list[0;4;2]
- a dictionary with complex values can also be viewed:
    - notationally: complex_dict["some_key"]["some_other_key"]
    - functionally as a multivaruate map: complex_dict["some_key";"some_other_key"]

### 6.5.3. Projection and index elision

- the notation for function projection and elided indices in a list are identical
- in a nested list (or a complex dictionary) the first index is go down (choosing from the row indices) and the second one is go over (choosing from the column indices)
    - ordinary positional retreival: if we omit the first index (the row index), we get all the elements in the specified column

### 6.5.4. Out of bounds index

- the behaviour of item indexing in case of out of bounds indices is also motivated by viewing lists as maps
- in traditional languages out of bound indexing results in an error or an exception:
    - in Python: IndexError
    - in Java: ArrayIndexOutOfBoundsException (for arrays) or IndexOutOfBoundsException (for lists)
- in q, a list is viewed as a map defined on a sub-domain of integers
    - but the domain of the (indexing?) function is extended to all integers by assigning null output values outside the defined domain, where null values mean missing values
- the behavior of dictionaries are completely analogous

## 6.6. Atomic functions

- q is a vector language
- this partially comes from the fact that all built-in operations are atomic:
    - atomic functions recurse into an arguments structure until it gets to atoms and apply there
    - atomic functions do not use loops or other contol flow constructs

### 6.6.1. Monadic atomic functions and "map"

- the output conforms to the input: the result of the application of a monadic atomic function always has the same shape as its argument
- when applying an atomic function to a list is the same as applying it to all of its elements
    - in traditional languages this is achieved with foreach or
    - in functional languages with the map() function

### 6.6.2. Dyadic atomic functions and "zip"

- a dyadic function can be atomic in either one or both of its arguments
    - atomic in one argument: fix the non-atomic argument, project on the atomic one. this can be thought of as a monadic atomic function
    - example: the find function (?) is atomic only its second argument, since it consumes the entire first argument in its search
        - (x?y means search for item y in list x and return the index of the item)

In [ ]:
10 20 30 10?10

- the arithmetic, comparison and relation operators are all atomic in both of their operands, which results in 4 cases of application:
    - atom with atom
    - atom with list
    - list with atom (same as atom with list?)
    - list with list (only works with lists of the same length)
- in traditional programming, "zip" results in similar behavior (zip() function in Python)

In [ ]:
list1:1 2 3 4 5
list2:10 20 30 40 50
int1:1
int2:2

In [ ]:
int1+int2
int1+list1
list1+int1
list1+list2

### 6.6.3. Creating atomic functions

- The composition of atomic functions is atomic. Hence,
    - one way to build custom atomic functions is to __compose built-in atomic functions__. First monadic.
    - (another way is to create advers???)

## 6.7. Adverbs

- Adverbs are higher-order functions that modify the behavior of functions (operators) for application on lists.
    - The terminology derives from thinking of q operators as verbs.
- Use case: __adverbs are used to apply non-atomic functions on lists without loops__
- Proficiency in the use of adverbs is one skill that separates q pretenders from q contenders.

### 6.7.1. Monadic each

- examples for non-atomic functions:
    - aggregate functions: count, sum, min, max, avg
        - and their atomic counterparts: count each, sums, mins, maxs, avgs
    - other non-atomic functions: reverse, enlist
- each is a general higher order function, which can be used as map() functions in functional languages:
    - reverse each a_list
    - enlist each a_list: creates a matrix from a list where each item becomes a singleton list in the matrix
        - flip enlist a_list does the same only faster

In [ ]:
show nlist:(1 2 3 4;10 20)
show nnlist:((101;102;(103;104));nlist)

In [ ]:
count each nlist
each[count;nlist] / this reveals how each is similar to map() functions in functional languages

In [ ]:
count nnlist
count each nnlist
(count each) each nnlist

In [ ]:
freq:(1;4;12;32;36;10;3;2)
sums(freq)

In [ ]:
freq2:(5;23;44;113;134;40;25;2)
freq2
100 * sums(freq2) % (sum freq2)
(`int$10000 * sums(freq2) % (sum freq2)) % 100

### 6.7.2. Each both __'__

- the dyadic each-both modifies a dyadic function (operator, verb) to apply pair-wise to corresponding items in the lists
- functions modified by the each both modifier has the same properties as dyadic atomic functions
    - throws a length error if the two arguments do not line up, but
    - extends an atom to match a list
    - however, it does not always work as expected on general lists
        - to make a list of pairs from a pair of lists: flip (L1;L2)
- applying each both on tables (with the same number of records), results a sideways join: the joined table will contain columns from both tables and will have the same number of rows

In [ ]:
strl1:("abc"; "uv")
strl2:("de"; "xyz")

In [ ]:
strl1,strl2

In [ ]:
strl1,'strl2

In [ ]:
,'[strl1;strl2] / in a functional form

In [ ]:
2#("abcde"; "fgh"; "ijklm")
2#'("abcde"; "fgh"; "ijklm")

In [ ]:
show L1:(enlist `a; `b)
L2:1 2
newL:L1,'L2
newL

In [ ]:
flip (L1;L2)
type flip (L1;L2)

### 6.7.3. Each left \\:

- The each left adverb modifies a dyadic funcition so that it applies the second argument with each item of the first argument

In [ ]:
somestr:("abc"; "de"; "f")

In [ ]:
somestr ,\: ">" / the left operand is always considered to be an atom even if it is a list

In [ ]:
somestr,' " />" / the second argument is taken as a list based on its type

### 6.7.4. Each right /:

- The each right adverb modifies a dyadic function so that to apply the first argument with each item of the second argument

In [ ]:
"<",/:somestr

In [ ]:
"<",'somestr

- combining the each right and each both

In [ ]:
"<",/:somestr,\:" />"

### 6.7.5. Cross product

- A cross product of two lists pairs each item on the left with each item on the right
    - it is an adverb of join composed with each left and each right (then razed to remove a level of nesting)
    - notation: cross, which can be written as raze list1,/:\:list2

In [ ]:
pre_cross:1 2 3,/:\:10 20

In [ ]:
pre_cross

In [ ]:
((count each) each) each pre_cross

In [ ]:
pre_cross[0;0;0]

In [ ]:
cross_pr:raze 1 2 3,/:\:10 20

In [ ]:
cross_pr

In [ ]:
cross_pr[0]

### 6.7.6 Over (/) for accumulation

- In functional programming languages, over is also called _fold_ or _reduce_
    - this is the other half of the _map-reduce_ paradigm (each-over)
- The over adverb is a higher-order function that provides the principal mechanism for recursion in q
- The over adverb, in its simplest form, modifies a dyadic function to accumulate results over a list

In [ ]:
0 +/ til 11

In [ ]:
imp_sum:{[x;y] 0N!(x;y); x+y} / the + operator expressed in an imperative way

In [ ]:
imp_sum[2;3]

In [ ]:
0 imp_sum/til 11

- To omit the unnecessary initial value, put the adverb in a paranthesis and make it a monadic function:
    - for summing the items in a list: (+/) this is an adverb in k programming languages
    - in q (+/) is equal to __sum__
    - (|/): maximum
    - (&/): minimum
    - (*/): prd

In [ ]:
(+/) til 11 / same as above only in a monadic form

In [ ]:
L1:1 2 3 4 5 77
L2: 54 32 64 11 14

In [ ]:
max[L1,L2]

In [ ]:
max[L1]|max[L2]

- ,/ (join over) applied over a list concatenates the items eliminating the top level of nesting

In [ ]:
(,/)((1 2 3; 4 5); (100 200; 300 400 500))
raze ((1 2 3; 4 5); (100 200; 300 400 500))

- custum functions can be used with over to make them atomic?
    - they can be used infix too

### 6.7.7. Iteration

- Another pattern of recursion without loops:
    - number of iterations function/ arguments to the function

In [ ]:
fib_elem:{x,sum -2#x}

In [ ]:
fib_elem 1 1

In [ ]:
10 fib_elem/1 1 / infix
fib_elem/[10, 1 1] / in a prefix form

- Yet another version of over runs the recursion until convergence, or until a loop is detected.
- How does q determine convergence?
    - At each step the result is compared to that of the previous step.
    - If the two agree within equality tolerance (10-14 at the time of this writing – Sep 2015) the algorithm is considered to have converged and the recursion is complete; otherwise it continues.
- How did q detect the cycle and stop the iteration?
    - At each step it compares the result with the initial value.
    - If they match, a cycle has occurred and the recursion is halted; otherwise it continues.
        - Note that we mean "match" literally as in the ~ operator.

__While loop__
- the final overload of the over adverb is the equivalent of the while loop
- It provides a declarative way to specify a test to end the iteration.
- Thinking functionally, we provide a predicate function that is applied to the result at each step. The iteration continues as long as the predicate result is 1b and stops otherwise. For example, we stop the computation of the Fibonacci sequence once it exceeds 1000 as follows.

In [ ]:
fib:{x,sum -2#x}
fib/[{1000>last x}; 1 1]

### 6.7.8 Scan \\

- The scan adverb is a higher-order function that behaves just like over except it returns all the intermediate accumulations instead of just the final one
    - Scan returns an output that has the same number of items as its input

In [ ]:
f:{-2+x*x}

In [ ]:
0+\1 2 3 4 5 6 7 8 9 10
(*\)1 2 3 4 5 6 7 8 9 10
(|\)7 8 4 3 10 2 1 9 5 6
(&\)7 8 4 3 10 2 1 9 5 6
10 f\1 2 3 4 5 6 7 8 9 10
(f\)1 2 3 4 5 6 7 8 9 10

### 6.7.9. Each previous (':)

- the adverb each previous provides a declarative way to provide a dyadic operation on each item of a list with its predecessor
- in a dyadic form, the left operand is the initializer value:

In [ ]:
5 -':5 4 3 2 1

- in a monadic form, there is no initializer value, it returns the first element unaltered:

In [ ]:
(-':)9 8 7 6 5 4

In [ ]:
deltas 9 8 7 6 5 4

### SUMMARY

- each
- each both ('):
    - joint: (,')
- each left (\:)
- each right (/:)
- over (/)
    - (+/) sum
- scan (\)
    - (+\) sums
- each previous (':)
    - (-':) deltas
    - (~':) differ

In [ ]:
(~':) 1 1 1 2 2 3 4 5 5 5 6 6
not (~':) 1 1 1 2 2 3 4 5 5 5 6 6
differ 1 1 1 2 2 3 4 5 5 5 6 6

- An idiom with differ: to cut a list along its identical values

In [ ]:
L:1 1 1 2 2 3 4 5 5 5 6 6
differ L / list of booleans where item differs from previous one
where differ L / indices of elements which are different from their predecessors
(where differ L) cut L / cut original list where different items start

In [ ]:
show runs:(where differ L) cut L / store runs
show ct:count each runs / store count of each run
runs where ct=max ct / find the runs of maximum length

- find runs of increasing or decreasing integer values

In [ ]:
L:9 8 7 11 10 12 13
(where -0W>':L) cut L
(where 0W<':L) cut L

## 6.8. General application (of functions)

- The syntacic forms, square brackets and juxtaposition (used with list indexing, key lookup and function application) are both syntactic sugars
- __q actually treats function application as a higher-order function that takes a function and value(s) and evaluates the function at the values__
- the simpler forms of general application are usually written infix and considered to be verbs

In [ ]:
fsq:{x*x}

In [ ]:
fsq[2 4]

### 6.8.1. The @ verb

- univalent mathematical mapping:
    - in case of lists: retreive a list item by index
    - dictionaries: looking up a value by a key
    - functions: evaluating a monadic function
        - @ applies a monadic mapping to an argument
- both infix and prefix notation is acceptable
- applying a niladic function with @:
    - niladic@(::)
    - @[niladic;::]

In [ ]:
GL: 10 20 30 40 50

In [ ]:
GL@1

In [ ]:
@[GL;3]

In [ ]:
gd:`a`b`c!11 22 33

In [ ]:
gd@`b

In [ ]:
@[gd;`a]

In [ ]:
gf:{x*x}

In [ ]:
gf@3

In [ ]:
@[gf;6]

- general application with a non-atomic argument
    - for lists, dictionaries and atomic functions, @ yields an output that conforms to the shape of the input
    - an aggregate function collapses the top level of nesting from the input
    - a uniform function has the same length of output as the input list

In [ ]:
t:([]c1:1 2 3; c2:`a`b`c)
t@1
d:`a`b`c!10 20 30
d@`b
kt:([k:`a`b`c] v:1.1 2.2 3.3)
kt@`c

6.8.2. Verb Dot .

- Indexing a list at depth, retrieving a nested value from a dictionary and evaluating a function with multiple parameters are all instances of applying a __multi-variate mapping__.
- The higher-order function . is the true form of multi-variate application in q.
- It applies a multi-variate mapping to multiple arguments and can be written infix or prefix.

In [ ]:
gln:((2 3 4);(6 7 9))

In [ ]:
gln[0][1]

In [ ]:
gln[0;1]

In [ ]:
gln . 0 1

In [ ]:
gcd:`a`b`c!(10 20 30; 40 50; enlist 60)

In [ ]:
gcd . (`c;0)

In [ ]:
gmvf:{x*y}

- the dot verb allows us to apply a multi-variate function to a list of arguments instead of multiple individual arguments
    - this means that the function is defined on a vector instead of its indivudial components
    - use case: dynamic dispatch: where we cannot know the valence of the function in advance
        - similar to *args in Python and ... in Java
- you can apply a monadic function with the dot verb, only you have to enlist its only argument to be a singleton vector
    - thus, the dot verb makes the @ verb redundant in q

In [ ]:
gmvf . 33 22

- to denote an elided index with ., use nil instead of empty index

In [ ]:
gln . (::;1)

In [ ]:
GL . enlist 0

In [ ]:
t:([]c1:1 2 3;c2:`a`b`c)
t . (1; `c2)

In [ ]:
kt:([k:`a`b`c] v:1.1 2.2 3.3)
kt . `b`v

In [ ]:
kt

- indexing at depth with a table with nested field values

In [ ]:
t:([] c1:`a`b`c; c2:(1 2; 100 200 400; enlist 1000))
t . (1; `c2)
t . (1; `c2; 1)

### 6.8.3. General form of verb application

- dot and @ verbs written in prefix form:

In [ ]:
L:10 20 30 40 50
@[L; 1]
@[L; 0 2]
m:(10 20 30; 100 200 300)
.[m;0 2] / note the lack of semi-colons between the two indices

### 6.8.4. General apply (@) with monadic functions

- general apply: applies a function on the sub-domain of the data structure
- notation: @[data_structure;indices;function_to_apply]
- this allows the result to be chained into further operations
- general application of @ accurs along a sub-domain at the top-level (HOW TO REACH DOWN?)

- The syntax for general application of a monadic atomic function on a list is,
    - @[L;I;f]
    - where L is the list and I is a collection of indices into L and f is the function to apply on I sub-domain of L.
- Viewing L as a mapping, I is a top-level sub-domain of L. In fact, this form generalizes to any data structure viewed as a mapping. 

In [ ]:
L:10 20 30 40 50

In [ ]:
@[L;2 4;neg] / general application returns the entire input data structure with the specified item changed

In [ ]:
neg L@2 / normal applicaton only returns the selected item

- to modify the original data structure in-place, use pass-by-name

In [ ]:
L

In [ ]:
@[`L;2;neg]

In [ ]:
L

### 6.8.5. General apply (@) dyadic functions

- the shape of the supplied operand must conform to the specified sub-domain of the input data

- The general form of functional @ for a dyadic atomic function is:
    - @[L; I; g; v]
    - where I is a top-level sub-domain of L; g is a dyadic function; and v is an atom or list conforming to I.
- example use case: assigning multiple values to multiple items in a list
- in-place modification works the same as with the genral apply on monadic functions

In [ ]:
@[L;1 4;+; 100 200]

### 6.8.6. General apply (.) for monadic functions

- In contrast with @, the vector argument of . reaches down into the data structure and picks out a single point in the domain. Here we target that point with a monadic function.
- The general form of . for monadic functions is:
    - .[L; I; f]
    - Here L is a data structure, I is an in-depth sub-domain of L and f is a monadic atomic function.

In [ ]:
m:(10 20 30; 100 200 300)
.[m;(0;1)]
.[m;(0;::)] / using nil, we apply the function on all items at that level
.[m;(0;::);neg]
d:`a`b`c!(10 20 30; 40 50; enlist 60)
.[d; (`a; 1)]

### 6.8.7. General apply (.) for dyadic functions

In [93]:
show mm:(10 20 30 40;100 200 300 400 500)

10 20 30 40
100 200 300 400 500


In [95]:
.[mm;(1;::);+;1 2 3 4 5]

10 20 30 40
101 202 303 404 505
